# Monte-Carlo propagation of uncertainties

This guide introduces the [mc](../reference/index.rst#monte-carlo) sub package for propagation of uncertainties using Monte-Carlo.

Monte-Carlo (MC) can propagate uncertainties through most operations including non-differentiable ones and operations that introduce correlations.
It does so by sampling new data from an input with uncertainties, performing the desired operation on each sample, and combining the results to compute, e.g., a mean and variance.
While this method is powerful, it is very expensive and in practice only applicable to relatively small and well behaved data.

In math terms, the `mc` package does the following.
The examples below will make the terminology more concrete.

Given a measured, empirical distribution of random variables $\mathbf{X}$, we want to compute some parameter $\theta$ (the result of our operation).
We estimate $\theta$ using an estimator $s$, such that $\hat{\theta} = s(\mathbf{X})$.
We assume all $X_i$ to be **independently** distributed with distribution $X_i = P(\mu_i, \sigma_i)$, where $\mu_i$ and $\sigma_i$ are the mean and standard deviation of $X_i$.
We draw $R$ samples ('replicas') from $P$:
$$
\mathbf{x}^\ast = [x_{i_1}^\ast,\, \dots, x_{i_n}^\ast], \quad \text{where} \quad x_i^\ast \sim X_i = P(\mu_i, \sigma_i).
$$
Then we pass each sample to the estimator $s$ to obtain replicas of the target parameter $\hat{\theta}^\ast = s(\mathbf{x}^\ast)$.
The final results are then, typically, the mean and variance over all $R$ replicas
$$
\mu(\hat{\theta}^\ast) = \frac{1}{R} \sum_{r=1}^{R}\,s(\mathbf{x}^\ast_r),\\\\
\text{var}(\hat{\theta}^\ast) = \frac{1}{R-1} \sum_{r=1}^{R}\,{\big(s(\mathbf{x}^\ast_r) - \mu(\hat{\theta}^\ast)\big)}^2
$$

Let's look at some examples to make this more concrete.
Suppose we are given some positions and time information and want to compute the corresponding speeds.
We will assume that all input data is independently normally distributed.
First, generate some dummy data of positions with uncertainties:

In [ ]:
from typing import Dict

import numpy as np
import plopp as pp
import scipp as sc

from scippuncertainty import mc

pp.patch_scipp()
%matplotlib widget

rng = np.random.default_rng(3781)
n = 100
x = sc.linspace("x", 1, 2, n)
variances = rng.uniform(0.01, 0.1, n)
pos = sc.DataArray(
    sc.array(
        dims=["x"],
        values=x.values + rng.normal(0, np.sqrt(variances)),
        variances=variances,
        unit="m",
    ),
    coords={"x": x},
)
pos.plot()

Configure ScippUncertaintie's logger to get some extra output during this guide.

In [ ]:
import logging
from scippuncertainty.logging import get_logger
handler = logging.StreamHandler()
handler.setLevel("INFO")
get_logger().addHandler(handler)
get_logger().setLevel("INFO")

## Equivalent to regular uncertainty propagation

In this example, use an array of times without uncertainties:

In [ ]:
time = sc.DataArray(sc.linspace("x", 0.1, 10.0, n, unit="s"), coords=pos.coords)

And define a function to compute the speed.
This corresponds to the estimator $s$.
(You will see below why this returns a `dict`.)

In [ ]:
def compute_speed(pos: sc.DataArray, time: sc.DataArray) -> Dict[str, sc.DataArray]:
    return {"speed": pos / time}

Given these times and speed calculation, we could do regular error propagation since the input is normally distributed.
So we can use this to check if our MC results makes sense.

In [ ]:
speed_regular = compute_speed(pos=pos, time=time)["speed"]

Now, in order to compute the uncertainties with MC, we need to create a few helper objects.
First, define a sampler.
This will be used to draw new samples from the input `pos`.
Since we assume normally distributed data, we use [NormalDenseSampler](../generated/modules/scippuncertainty.mc.sampler.NormalDenseSampler.rst).
This defines the distribution $X_i = P(\mu_i, \sigma_i)$.

In [ ]:
pos_sampler = mc.NormalDenseSampler(pos)

Next, we need to define how to collect the replicas and compute an output statistic.
In this case, we simply want to compute the mean and variance, so we use [VarianceAccum](../generated/modules/scippuncertainty.mc.accumulator.VarianceAccum.rst).

In [ ]:
accumulator = mc.VarianceAccum()

Finally, we can use [mc.run](../generated/modules/scippuncertainty.mc.driver.run.rst) to put everything together and actually perform the MC computation.

We pass a `dict` to the samplers that identifies the `pos_sampler` with the `pos` argument of `compute_speed`.
The accumulators `dict` defines how to accumulate each output of `compute_speed`.
There is only one, but we still have to match the name in the `dict`s  returned by the function with the accumulators.
Since `time` has no uncertainties, we simply bind our fixed values using `partial`.

`n_samples` corresponds to the number of replicas $R$.
It is very high in this case because the computation is quite fast.
In practice, numbers in the hundreds are more feasible.
Lastly, we disable progress reporting as this does not work reliably in Jupyter.

In [ ]:
from functools import partial

results = mc.run(
    partial(compute_speed, time=time),
    samplers={"pos": pos_sampler},
    accumulators={"speed": accumulator},
    n_samples=10000,
    progress=False,
)
speed_mc = results["speed"]

Note the log message about the random seed emitted by `mc.run`.
You need to save this seed if you want to ever reproduce the calculation.

Now compare the results of the two calculations.
It looks like MC and 'regular' uncertainty propagation are in agreement.

In [ ]:
speed_mc.coords["x"] += 0.01
pp.plot({"regular": speed_regular, "mc": speed_mc}, norm="log")

Also compare the relative errors of both results.
Again, there is general agreement as expected.
But there are some deviations because there are not enough MC samples to get a higher precision.

<div class="alert alert-warning">
    
**Attention**
    
Always make sure that your MC has properly converged to the desired precision.
</div>

In [ ]:
def relative_error(da: sc.DataArray) -> sc.DataArray:
    return sc.stddevs(da) / abs(sc.values(da))

pp.plot({"regular": relative_error(speed_regular), "mc": relative_error(speed_mc)})

## 2

In [ ]:
time = sc.scalar(2.6, variance=0.4, unit="m")

In [ ]:
speed_regular = compute_speed(pos=pos, time=time)["speed"]

In [ ]:
speed_regular = compute_speed(pos=pos,
                              time=sc.broadcast(time, sizes=pos.sizes).copy())["speed"]

In [ ]:
pos_sampler = mc.NormalDenseSampler(pos)
time_sampler = mc.NormalDenseSampler(time)

In [ ]:
accumulator = mc.VarianceAccum()

In [ ]:
results = mc.run(
    compute_speed,
    samplers={"pos": pos_sampler, "time": time_sampler},
    accumulators={"speed": accumulator},
    n_samples=10000,
    progress=False,
)
speed_mc = results["speed"]

In [ ]:
pp.plot({"regular": relative_error(speed_regular), "mc": relative_error(speed_mc)})

## 3 cov

In [ ]:
pos_sampler = mc.NormalDenseSampler(pos)
time_sampler = mc.NormalDenseSampler(time)

In [ ]:
variance_accumulator = mc.VarianceAccum()
covariance_accumulator = mc.CovarianceAccum(dims=("x0", "x1"))

In [ ]:
def compute_speed(pos: sc.DataArray, time: sc.DataArray) -> Dict[str, sc.DataArray]:
    speed = pos / time
    return {"speed": speed, "speed_cov": speed}

In [ ]:
results = mc.run(
    compute_speed,
    samplers={"pos": pos_sampler, "time": time_sampler},
    accumulators={"speed": variance_accumulator,
                  "speed_cov": covariance_accumulator},
    n_samples=10000,
    progress=False,
)
speed_mc = results["speed"]
speed_cov = results["speed_cov"]

In [ ]:
speed_cov.plot()

In [ ]:
def pearson_correlation(cov: sc.DataArray) -> sc.DataArray:
    std0 = sc.sqrt(sc.array(dims=[cov.dims[0]], values=np.diag(cov.values)))
    std1 = std0.rename({cov.dims[0]: cov.dims[1]})
    return cov / (std0 * std1)

In [ ]:
pearson_correlation(speed_cov).plot(vmin=0.0, vmax=1.0)

In [ ]:
speed_cov